In [ ]:
import random
import numpy as np
import pandas as pd
import os
import sys
import inspect
import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

In [ ]:
# set current notebook path
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [ ]:
from transformers import BertConfig, BertTokenizer
from nltk.tokenize import word_tokenize

from NER_Module import load_pretrained_model, BertNerModel, NerGritDataset, NerDataLoader

In [ ]:
# import nltk

# nltk.download('punkt')

## Training mode

In [ ]:
## datapath 
data = "F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\NER_Module\\test\\data"
train_file = 'shopee_train.txt'
validation_file ='shopee_test.txt'
#test_file = 'test_file.txt'
model_path = 'F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\model'

In [ ]:
train_data_path = os.path.join(data,train_file)
validation_data_path = os.path.join(data,validation_file)
#test_data_path = os.path.join(data,test_file)

In [ ]:
## load pretrained model
base_model,tokenizer = load_pretrained_model(num_labels=NerGritDataset.NUM_LABELS, is_cuda=True)

In [ ]:
base_model.device

In [ ]:
# load data
train_dataset = NerGritDataset(train_data_path, tokenizer, lowercase=True)
valid_dataset = NerGritDataset(validation_data_path, tokenizer, lowercase=True)
#test_dataset = NerGritDataset(test_data_path, tokenizer, lowercase=True)

train_loader = NerDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=True)  
valid_loader = NerDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)  
#test_loader = NerDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [ ]:
NerGritDataset.NUM_LABELS

In [ ]:
w2i, i2w = NerGritDataset.LABEL2INDEX, NerGritDataset.INDEX2LABEL
# print(w2i)
# print(i2w)

In [ ]:
print(type(i2w))

In [ ]:
## model initialisation
bert_model = BertNerModel(num_classes=NerGritDataset.NUM_LABELS, i2w=i2w, exp_id='test_3', random_state=33, device='cuda')

In [ ]:
bert_model.train(train_loader=train_loader, val_loader=valid_loader, model_dir=model_path, num_epochs=10, 
                 evaluate_every=2, early_stop=3, valid_criterion='F1')

In [ ]:
result_df = bert_model.predict(test_loader=test_loader, save_path= model_path, filename='result_test_2')

## Load model and predict (No training)

In [ ]:
## model_path 
model_folder = './'
filename = 'best_model_test_2.th'
model_path = os.path.join(model_folder,filename)

In [ ]:
w2i, i2w = NerGritDataset.LABEL2INDEX, NerGritDataset.INDEX2LABEL

In [ ]:
## model initialisation
bert_predict_model = BertNerModel(num_classes=NerGritDataset.NUM_LABELS, i2w=i2w, exp_id='test_2', random_state=33, device='cuda')

In [ ]:
## load custom model
bert_predict_model, bert_predict_tokenizer=bert__predict_model.load_modellocal(path=model_path, load_tokenizer= True)

In [ ]:
## load test_data
predict_dataset = NerGritDataset(test_data_path, bert_predict_tokenizer, lowercase=True)

In [ ]:
## load data trainer
predict_loader = NerDataLoader(dataset=predict_dataset, max_seq_len=512, batch_size=16, num_workers=16, shuffle=False)

In [ ]:
bert_predict_model.device

In [ ]:
result_path ='F:\\Bala_EU_DSVM_BACKUP\\Shopee_challenge\\Shopee_Challenge\\model'

In [ ]:
predict_df = bert_predict_model.predict(test_loader=train_loader, save_path= None, filename='predict_test_2')

In [ ]:
## Compare two dataframes are equal 
predict_df.equals(result_df)

In [ ]:
len(train_loader)

In [ ]:
predict_df.head()